## Importing libraries

In [ ]:
# importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns

# to show full rows and columns
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# disable the warnings
import warnings
warnings.filterwarnings('ignore')

# display upto 2 decimals
pd.options.display.float_format = "{:,.3f}".format

# importing libraries for ML algorithm
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score, mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# reading csv data
data = pd.read_csv('train.csv', header='infer', index_col='Id')

## 1. Data Understanding

In [ ]:
# checking the shape of DF
data.shape

In [ ]:
# inspecting loaded data
data.head(5)

In [ ]:
# checking null values count
data.isnull().sum()

In [ ]:
# checking dtypes and other info
data.info()

In [ ]:
# checking int & float dtypes and their values
data.describe()

## 2. Data Visualization

### 2.1 Univariate Analysis before treating missing values

In [ ]:
# function to generate the univariate count plots for all the object data types
def createplot_count(v1,v2,v3):
    gs  = gridspec.GridSpec(4, 1, height_ratios=[1, 1 ,1.5, 1])
    plt.subplot2grid((30,3),(v1,v2))
    sns.countplot(data[v3])
    plt.title(v3, fontsize=18, fontweight='bold')
    plt.xticks(rotation=90, fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel('', fontsize=14)
    plt.ylabel('Count', fontsize=14)
    
# function to generate the univariate distribution plots for all the numerical data types
def createplot_dist(v1,v2,v3):
    gs  = gridspec.GridSpec(4, 1, height_ratios=[1, 1 ,1.5, 1])
    plt.subplot2grid((30,2),(v1,v2))
    sns.distplot(data[v3], hist=True)
    plt.title(v3, fontsize=18, fontweight='bold')
    plt.xticks(rotation=90, fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel('', fontsize=14)
    plt.ylabel('Density', fontsize=14)
    
# selecting all the object types features for univariate analysis
objlist = data.select_dtypes(include='object').columns.tolist()

# Some numerical columns which has a count or rating
Xlist = ['OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr',\
        'KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars']

# selecting all the numerical types features for univariate analysis
numlist = data.select_dtypes(include=['int64','float64']).columns.tolist()

numlist = list(set(numlist)-set(Xlist))

In [ ]:
# generating univariate plots for all the object data types

i = 0
plt.figure(figsize=(20,300))
for j in range(21):
    for k in range(3):
        if i == len(objlist):
            break
        else:    
            createplot_count(j,k, objlist[i])
            i+=1
            
plt.show()   

In [ ]:
# generating univariate plots for the above mentioned data types which i want to keep as numerical as it shows
# an order, otherwise i will have to create extra dummies for these features

i = 0
plt.figure(figsize=(20,250))

for j in range(21):
    for k in range(3):
        if i == len(Xlist):
            break
        else:    
            createplot_count(j,k, Xlist[i])
            i+=1
            
plt.show() 

In [ ]:
# generating univariate distribution plots for all the continuous numerical data types

i = 0
plt.figure(figsize=(20,300))
for j in range(21):
    for k in range(2):
        if i == len(numlist):
            break
        else:    
            createplot_dist(j,k, numlist[i])
            i+=1

### 2.2 Missing Values Treatment

In [ ]:
# dropping duplicates if any
data = data.drop_duplicates()

In [ ]:
# features that needs to converted to object type
obj1 = ['MSSubClass','MoSold','YrSold']

data[obj1] = data[obj1].astype(str)

In [ ]:
# Alley: Type of alley access to property
print(f"Null Value count: \n{data['Alley'].isnull().value_counts()}")

# updating null with NA (No alley access)
data.loc[data['Alley'].isnull(), ['Alley']] = 'NA'

In [ ]:
# MasVnrType: Masonry veneer type
print(f"Null Value count: \n{data['MasVnrType'].isnull().value_counts()}\n")
print(f"Mode Value: {data['MasVnrType'].mode()}")

# updating null with mode values 
data.loc[data['MasVnrType'].isnull(), ['MasVnrType']] = 'None'

In [ ]:
# BsmtQual: Evaluates the height of the basement, BsmtCond: Evaluates the general condition of the basement
# BsmtExposure: Refers to walkout or garden level walls, BsmtFinType1: Rating of basement finished area
# BsmtFinType2: Rating of basement finished area (if multiple types)

print(f"Null Value count: \n{data['BsmtQual'].isnull().value_counts()}\n")
print(f"Null Value count: \n{data['BsmtCond'].isnull().value_counts()}\n")
print(f"Null Value count: \n{data['BsmtExposure'].isnull().value_counts()}\n")
print(f"Null Value count: \n{data['BsmtFinType1'].isnull().value_counts()}\n")
print(f"Null Value count: \n{data['BsmtFinType2'].isnull().value_counts()}\n")

# updating missing values with NA as the missing data represents No basement in the property

data.loc[data['BsmtQual'].isnull(), ['BsmtQual']] = 'NA'
data.loc[data['BsmtCond'].isnull(), ['BsmtCond']] = 'NA'
data.loc[data['BsmtExposure'].isnull(), ['BsmtExposure']] = 'NA'
data.loc[data['BsmtFinType1'].isnull(), ['BsmtFinType1']] = 'NA'
data.loc[data['BsmtFinType2'].isnull(), ['BsmtFinType2']] = 'NA'

In [ ]:
# Electrical: Electrical system

print(f"Null Value count: \n{data['Electrical'].isnull().value_counts()}\n")
print(f"Mode Value: {data['Electrical'].mode()}")

# updating null with mode values 
data.loc[data['Electrical'].isnull(), ['Electrical']] = 'SBrkr'

In [ ]:
# Fireplaces: Number of fireplaces & FireplaceQu: Fireplace quality

print(f"Null Value count: \n{data['FireplaceQu'].isnull().value_counts()}\n")

# Almost 47% properties have no fireplaces, so this is the reason that 47% of fireplace quality is null
data['Fireplaces'].value_counts(normalize=True)*100

# Lets update null values with 'NA' for this feature as it means No fireplace in the property
data.loc[data['FireplaceQu'].isnull(), ['FireplaceQu']] = 'NA'

In [ ]:
# GarageType: Garage location, GarageFinish: Interior finish of the garage, GarageQual: Garage quality
# GarageCond: Garage condition

print(f"Null Value count: \n{data['GarageType'].isnull().value_counts()}\n")
print(f"Null Value count: \n{data['GarageFinish'].isnull().value_counts()}\n")
print(f"Null Value count: \n{data['GarageQual'].isnull().value_counts()}\n")
print(f"Null Value count: \n{data['GarageCond'].isnull().value_counts()}\n")

# Lets update null values with 'NA' for this feature as it means No garage in the property
data.loc[data['GarageType'].isnull(), ['GarageType']] = 'NA'
data.loc[data['GarageFinish'].isnull(), ['GarageFinish']] = 'NA'
data.loc[data['GarageQual'].isnull(), ['GarageQual']] = 'NA'
data.loc[data['GarageCond'].isnull(), ['GarageCond']] = 'NA'

In [ ]:
# PoolQC: Pool quality
# PoolArea

print(f"PoolQC Null Value count: \n{data['PoolQC'].isnull().value_counts(normalize=True)*100}\n")

# let's update null values with 'NA' which means no pool
data.loc[data['PoolQC'].isnull(), ['PoolQC']] = 'NA'

In [ ]:
# Fence: Fence quality
# Lets update null values with 'NA' for this feature as it means No fence or Miscellaneous feature in the property

print(f"Null Value count: \n{data['Fence'].isnull().value_counts()}\n")
data.loc[data['Fence'].isnull(), ['Fence']] = 'NA'

# MiscFeature: Miscellaneous feature not covered in other categories

print(f"Null Value count: \n{data['MiscFeature'].isnull().value_counts()}\n")
data.loc[data['MiscFeature'].isnull(), ['MiscFeature']] = 'NA'

In [ ]:
# GarageYrBlt
print(data[['GarageYrBlt','YearBuilt']].corr())

# we can see that almost 70% of the Garage year built matches the YearBuilt, it makes sense to impute it for missing values
data.loc[data['GarageYrBlt'].isnull(), ['GarageYrBlt']] = data['YearBuilt']
data['GarageYrBlt'] = data['GarageYrBlt'].astype(int)

In [ ]:
# LotFrontage: Linear feet of street connected to property

data['LotFrontage'].median()

# There are some null values which we need to impute, but its observed that 99% of the null values are 
# because there is no alley access to those properties// Out of 259, 254 is null where Alley=NA
len(data[(data['LotFrontage'].isnull()) & (data['Alley'] =='NA')].index)

data.loc[data['LotFrontage'].isnull(), 'LotFrontage'] = 0

# checking distribution
data['LotFrontage'].hist()
plt.show()

In [ ]:
#MasVnrArea: Masonry veneer area in square feet

data[data['MasVnrType']=='None'][['MasVnrArea','MasVnrType']]

# Its observed that for MasVnrType 'None' value ,MasVnrArea is 0
data.loc[(data['MasVnrArea'].isnull())&(data['MasVnrType']=='None'), 'MasVnrArea'] = 0

# checking distribution
data['MasVnrArea'].hist()
plt.show()

### 2.3 Univariate Analysis after treating missing values

In [ ]:
# selecting all the object types features for univariate analysis
objlist = data.select_dtypes(include='object').columns.tolist()

# Some numerical columns which has a count or rating
Xlist = ['OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr',\
        'KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars']

# selecting all the numerical types features for univariate analysis
numlist = data.select_dtypes(include=['int64','float64']).columns.tolist()

numlist = list(set(numlist)-set(Xlist))

In [ ]:
# generating univariate plots for all the object data types
i = 0
plt.figure(figsize=(20,300))
for j in range(21):
    for k in range(3):
        if i == len(objlist):
            break
        else:    
            createplot_count(j,k, objlist[i])
            i+=1
            
plt.show()            

In [ ]:
# generating univariate plots for the above mentioned data types which i want to keep as numerical as it shows
# an order, otherwise i will have to create extra dummies for these features
i = 0
plt.figure(figsize=(20,250))

for j in range(21):
    for k in range(3):
        if i == len(Xlist):
            break
        else:    
            createplot_count(j,k, Xlist[i])
            i+=1
            
plt.show()            

In [ ]:
# generating univariate distribution plots for all the continuous numerical data types
i = 0
plt.figure(figsize=(20,300))
for j in range(21):
    for k in range(2):
        if i == len(numlist):
            break
        else:    
            createplot_dist(j,k, numlist[i])
            i+=1

### 2.4 Bivariate and multivariate Analysis

In [ ]:
# plotting Bivariate & Multivariate plots
plt.figure(figsize=(18,120))

plt.subplot2grid((10,2),(0,0))
sns.scatterplot(x=data['YearBuilt'], y=data['SalePrice'])
plt.title('Year Built vs Price', fontsize=20, fontweight='bold')
plt.ylabel('Sale Price', fontsize=14); plt.xlabel('Year Built',fontsize=14)
plt.yticks(fontsize=12); plt.xticks(rotation=90,fontsize=12)

plt.subplot2grid((10,2),(0,1))
sns.scatterplot(x=data['TotalBsmtSF'], y=data['SalePrice'], hue=data['BsmtQual'])
plt.title('Basement Area(Quality) vs Price', fontsize=20, fontweight='bold')
plt.ylabel('Sale Price', fontsize=14); plt.xlabel('Basement Area',fontsize=14)
plt.yticks(fontsize=12); plt.xticks(rotation=90,fontsize=12)

plt.subplot2grid((10,2),(1,0))
sns.scatterplot(x=data['LotArea'], y=data['SalePrice'], hue=data['LotShape'])
plt.title('Lot Area(Shape) vs Price', fontsize=18, fontweight='bold')
plt.ylabel('Sale Price', fontsize=14); plt.xlabel('Lot Area',fontsize=14)
plt.yticks(fontsize=12); plt.xticks(rotation=90,fontsize=12)

plt.subplot2grid((10,2),(1,1))
sns.scatterplot(x=data['GrLivArea'], y=data['SalePrice'])
plt.title('Ground Living Area vs Price', fontsize=18, fontweight='bold')
plt.ylabel('Sale Price', fontsize=14); plt.xlabel('Ground Living Area',fontsize=14)
plt.yticks(fontsize=12); plt.xticks(rotation=90,fontsize=12)

plt.subplot2grid((10,2),(2,0))
sns.barplot(x=data['MSSubClass'], y=data['SalePrice'])
plt.title('Sub Class vs Price', fontsize=18, fontweight='bold')
plt.ylabel('Sale Price', fontsize=14); plt.xlabel('Sub Class',fontsize=14)
plt.yticks(fontsize=12); plt.xticks(rotation=90,fontsize=12)

plt.subplot2grid((10,2),(2,1))
sns.barplot(x=data['Neighborhood'], y=data['SalePrice'])
plt.title('Neighborhood vs Price', fontsize=18, fontweight='bold')
plt.ylabel('Sale Price', fontsize=14); plt.xlabel('Neighborhood',fontsize=14)
plt.yticks(fontsize=12); plt.xticks(rotation=90,fontsize=12)

plt.subplot2grid((10,2),(3,0))
sns.boxplot(x=data['YrSold'], y=data['SalePrice'], hue=data['OverallQual'])
plt.title('Year Sold(Quality) vs Price', fontsize=20, fontweight='bold')
plt.ylabel('Sale Price', fontsize=14); plt.xlabel('Year Sold',fontsize=14)
plt.yticks(fontsize=12); plt.xticks(rotation=90,fontsize=12)

plt.subplot2grid((10,2),(3,1))
sns.boxplot(x=data['YrSold'], y=data['SalePrice'], hue=data['OverallCond'])
plt.title('Year Sold(Condition) vs Price', fontsize=20, fontweight='bold')
plt.ylabel('Sale Price', fontsize=14); plt.xlabel('Year Sold',fontsize=14)
plt.yticks(fontsize=12); plt.xticks(rotation=90,fontsize=12)

plt.subplot2grid((10,2),(4,0))
sns.barplot(x=data['HouseStyle'], y=data['SalePrice'], hue=data['BldgType'])
plt.title('HouseStyle(Building Type) vs Price', fontsize=18, fontweight='bold')
plt.ylabel('Sale Price', fontsize=14); plt.xlabel('HouseStyle',fontsize=14)
plt.yticks(fontsize=12); plt.xticks(rotation=90,fontsize=12)

plt.subplot2grid((10,2),(4,1))
sns.boxplot(x=data['Street'], y=data['SalePrice'], hue=data['MSZoning'])
plt.title('Zone classification Street vs Price', fontsize=18, fontweight='bold')
plt.ylabel('Sale Price', fontsize=14); plt.xlabel('Street',fontsize=14)
plt.yticks(fontsize=12); plt.xticks(rotation=90,fontsize=12)

plt.subplot2grid((10,2),(5,0))
sns.barplot(x=data['RoofStyle'], y=data['SalePrice'], hue=data['RoofMatl'])
plt.title('Roof Style(Roof Material) vs Price', fontsize=18, fontweight='bold')
plt.ylabel('Sale Price', fontsize=14); plt.xlabel('Roof Style',fontsize=14)
plt.yticks(fontsize=12); plt.xticks(rotation=90,fontsize=12)

plt.subplot2grid((10,2),(5,1))
sns.boxplot(x=data['GarageType'], y=data['SalePrice'], hue=data['GarageFinish'])
plt.title('Garage Type(Garage Finish) vs Price', fontsize=18, fontweight='bold')
plt.ylabel('Sale Price', fontsize=14); plt.xlabel('GarageType',fontsize=14)
plt.yticks(fontsize=12); plt.xticks(rotation=90,fontsize=12)

plt.subplot2grid((10,2),(6,0))
sns.boxplot(x=data['BsmtCond'], y=data['SalePrice'], hue=data['BsmtExposure'])
plt.title('Basement Condition(Basement Exposure) vs Price', fontsize=18, fontweight='bold')
plt.ylabel('Sale Price', fontsize=14); plt.xlabel('Basement Condition',fontsize=14)
plt.yticks(fontsize=12); plt.xticks(rotation=90,fontsize=12)

plt.subplot2grid((10,2),(7,0), colspan=2)
sns.heatmap(data[numlist].corr(),annot=True, cmap='Blues')
plt.title('Correlation Matrix 1', fontsize=18, fontweight='bold')
plt.xticks(rotation=45, fontsize=12)
plt.yticks(rotation=0, fontsize=12)

plt.subplot2grid((10,2),(8,0), colspan=2)
sns.heatmap(data[Xlist].corr(),annot=True, cmap='Blues')
plt.title('Correlation Matrix 2', fontsize=18, fontweight='bold')
plt.xticks(rotation=45, fontsize=12)
plt.yticks(rotation=0, fontsize=12)

plt.show()

In [ ]:
# plotting pairplot
data1 = data[['TotalBsmtSF','MasVnrArea','BsmtUnfSF','BsmtFinSF1','BsmtFinSF2','LotFrontage','GarageArea',\
      '1stFlrSF','2ndFlrSF','OpenPorchSF','WoodDeckSF','LotArea','GrLivArea']]

sns.pairplot(data1)
plt.show()

In [ ]:
# checking top 20 correlation's wrt SalePrice
data.corr()['SalePrice'][:20].sort_values(ascending=False)

## 3. Data Preparation
### 3.1 Dummy variables

In [ ]:
# dummy variables

# To reduce the number of dummies we can reduce the very less percentage categories 
# to a different category 'Others'

def create_other(v1):
    arr=[]
    for k, v in (data[v1].value_counts(normalize=True)*100).to_dict().items():
        if v < 2:
            arr.append(k)
        
    data.loc[data[v1].isin(arr), v1] = 'others' 

In [ ]:
# Calling the function after analyzing all the features values using valuecount percentages
# creating others category only for small percentage categories to reduce dummy variables

create_other('MSSubClass')
create_other('MSZoning')
create_other('Condition1')
create_other('Condition2')
create_other('RoofStyle')
create_other('RoofMatl')
create_other('Exterior1st')
create_other('Exterior2nd')
create_other('ExterCond')
create_other('Foundation')
create_other('Heating')
create_other('Electrical')
create_other('Functional')
create_other('GarageType')
create_other('GarageQual')
create_other('GarageCond')
create_other('MiscFeature')
create_other('SaleType')

In [ ]:
# selecting all the object types features for dummies creation
objlist_dummies = data.select_dtypes(include='object').columns.tolist()

# creating dummy variables
dummies_df = pd.get_dummies(data[objlist_dummies])

# concatenating dataframes
data = pd.concat([data, dummies_df], axis=1)
print(data.shape)

# dropping categorical value features
data = data.drop(objlist_dummies, axis=1)
print(data.shape)

### 3.2 Checking target variable distribution

In [ ]:
# plotting distribution plot for saleprice
plt.figure(figsize=(10,8))
sns.distplot((data['SalePrice']))
plt.show()

## As we can see that our target variable is not normally distributed, so we can perform some transformation and
# see which transformation can provide us with normally distributed target variable

In [ ]:
# plotting distribution plot for log(saleprice)
plt.figure(figsize=(10,8))
sns.distplot(np.log1p(data['SalePrice']))
plt.show()

## So, i chose log1p transformation as the saleprice values are very high but other feature values are not
# and since log1p works better than log for small values, we are going to use it for transformation and
# it has provided us with an almost normally distributed target variable

#### let's transform the target variable

In [ ]:
data['SalePrice'] = np.log1p(data['SalePrice'])

### 3.3 Outlier treatment

In [ ]:
# checking outliers

# selecting all the numerical types features for univariate analysis
numlist = data.select_dtypes(include=['int64','float64']).columns.tolist()

numlist = list(set(numlist)-set(Xlist))

data[numlist].describe(percentiles=[.1,.25,.5,.75,.9,.95,.99,.995,.999])

In [ ]:
# lets plot boxplot to see the outlier values after checking the outliers from above function

out_list=['TotalBsmtSF','MasVnrArea','BsmtFinSF2','BsmtFinSF1','MiscVal','1stFlrSF',\
          'OpenPorchSF','EnclosedPorch','LotArea','3SsnPorch']

plt.figure(figsize=(16,8))
sns.boxplot(x="variable", y="value", data=pd.melt(data[out_list]))
plt.show()

In [ ]:
# treating the outliers and removing features with more than 90% zero values

# we can see some significant ouliers in lotArea which can create problem with our log transformation of 
# independent variables, so its better to cap it at 99 percentile
data = data[data['LotArea']<=37567.640]

# Since 96% MiscVal data values are 0, its not going to give us anything significant and its better to drop it
data['MiscVal'].value_counts(normalize=True)*100
data = data.drop('MiscVal', axis=1)

# Since 98% 3SsnPorch data values are 0, its not going to give us anything significant and its better to drop it
data['3SsnPorch'].value_counts(normalize=True)*100
data = data.drop('3SsnPorch', axis=1)

# Since 98% LowQualFinSF data values are 0, its not going to give us anything significant and its better to drop it
data['LowQualFinSF'].value_counts(normalize=True)*100
data = data.drop('LowQualFinSF', axis=1)

# Since 92% ScreenPorch data values are 0, its not going to give us anything significant and its better to drop it
data['ScreenPorch'].value_counts(normalize=True)*100
data = data.drop('ScreenPorch', axis=1)

# Since 99.5% values of PoolArea are 0, it's better to drop these features 
data['PoolArea'].value_counts(normalize=True)*100
data = data.drop(['PoolArea'], axis=1)

In [ ]:
# lets plot boxplot to see the outlier values after checking the outliers from above function

out_list=['TotalBsmtSF','MasVnrArea','BsmtFinSF2','BsmtFinSF1','1stFlrSF','OpenPorchSF','EnclosedPorch']

plt.figure(figsize=(16,8))
sns.boxplot(x="variable", y="value", data=pd.melt(data[out_list]))
plt.show()

### 3.4 Splitting the data into train and test df

In [ ]:
# splitting the data into train and test dataframes using sklearn.model_selection.train_test_split method

train_df, test_df = train_test_split(data, train_size=0.7, test_size=0.3, random_state=10)

# checking shapes after splitting the data
print(train_df.shape)
print(test_df.shape)

### 3.5 Scaling

#### We have fixed some ouliers and all of them seems natural outliers. Let's apply scaling to all the variables including our target variable to which we applied log1p transformation. And if we don't get the desired results, we will transform all the variables with log1p transformation for scaling instead of MinMaxScaling

In [ ]:
# Log transformation to independent variables

# getting list of numerical columns for applying transformation
numlist1 = data.select_dtypes(include=['int64','float64']).columns.tolist()

# Initialising the scaler
scaler = MinMaxScaler()

# fitting the scaler and transforming the training data
train_df[numlist1] = scaler.fit_transform(train_df[numlist1])

# only transforming the test data
test_df[numlist1] = scaler.transform(test_df[numlist1])

# resetting the index
train_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

# We will use it if we don't get the desired good results from MinMaxScaling by applying log transformation 
# (we can use np.expm1 to reverse this transformation)
# data[numlist1] = data[numlist1].apply(lambda var: np.log1p(var))

In [ ]:
train_df[numlist1].describe()

In [ ]:
test_df[numlist1].describe()

## 4. Model Building

In [ ]:
# creating seperate independent and dependent variable df's

# training data
y_train = train_df.pop('SalePrice')
X_train = train_df

# test data
y_test = test_df.pop('SalePrice')
X_test = test_df

In [ ]:
# reshaping the dependent vriable df

y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)

### 4.1 Ridge Regression
#### Using RFE to get the important features using estimator=Ridge

In [ ]:
# using RFE to select features
ridge = Ridge()
ridge.fit(X_train, y_train)

# running the RFE, keeping the desired number of features to be half of the actual count of features
rfe = RFE(ridge,130)
rfe.fit(X_train, y_train)

In [ ]:
# creating df and extracting the features list

rfe_df = pd.DataFrame(list(zip(X_train.columns,rfe.support_,rfe.ranking_)),
             columns=('col','rfe support', 'rfe ranking')).sort_values(by='rfe ranking')

features_list = rfe_df[rfe_df['rfe support']==True]['col'].tolist()

#### Ridge Model with and without RFE selected features

In [ ]:
# finding the Ridge hyperparameter- best value of alpha needed for ridge model

# alpha values were selected from very low to higher value and after getting the best alpha, some more values 
# were inserted to make sure we get the best value

X_train_ridge_rfe = X_train[features_list]
X_test_ridge_rfe = X_test[features_list]

folds = [5, 10]
param = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1,
                   1.2, 1.25, 1.3, 1.35, 1.4, 1.5, 1.55, 1.6, 1.65, 1.7, 1.8, 1.9, 2.0, 2.05, 2.1, 2.15,
                   2.2, 3.0, 3.5, 3.75, 4.0, 4.25, 4.5, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000]}

model_cv = GridSearchCV(estimator=ridge, param_grid=param, scoring='neg_mean_absolute_error',
                       cv=folds[0], return_train_score=True, verbose=1)

model_cv_rfe_5 = GridSearchCV(estimator=ridge, param_grid=param, scoring='neg_mean_absolute_error',
                       cv=folds[0], return_train_score=True, verbose=1)

model_cv_rfe_10 = GridSearchCV(estimator=ridge, param_grid=param, scoring='neg_mean_absolute_error',
                       cv=folds[1], return_train_score=True, verbose=1)

model_cv.fit(X_train, y_train)
model_cv_rfe_5.fit(X_train_ridge_rfe, y_train)
model_cv_rfe_10.fit(X_train_ridge_rfe, y_train)

In [ ]:
# Printing the best hyperparameter alpha without feature selection
print(f'hyperparameter alpha without feature selection: {model_cv.best_params_}')

print(f'hyperparameter alpha with RFE feature selection and fold 5: {model_cv_rfe_5.best_params_}')

print(f'hyperparameter alpha with RFE feature selection and fold 10: {model_cv_rfe_10.best_params_}')

In [ ]:
# fitting the ridge model with alpha obtained above and checking the coefficients

alpha = model_cv.best_params_['alpha']
alpha_ridge_rfe_5 = model_cv_rfe_5.best_params_['alpha']
alpha_ridge_rfe_10 = model_cv_rfe_10.best_params_['alpha']

ridge = Ridge(alpha=alpha)
ridge_rfe_5 = Ridge(alpha=alpha_ridge_rfe_5)
ridge_rfe_10 = Ridge(alpha=alpha_ridge_rfe_10)

ridge.fit(X_train, y_train)
ridge_rfe_5.fit(X_train_ridge_rfe, y_train)
ridge_rfe_10.fit(X_train_ridge_rfe, y_train)

In [ ]:
# Coefficients of the models

Ridge_coeff = pd.concat([pd.DataFrame(np.reshape(ridge.coef_, (-1,1)), columns=['Ridge']), 
                         pd.DataFrame(np.reshape(ridge_rfe_5.coef_, (-1,1)), columns=['Ridge_RFE_5']),
                         pd.DataFrame(np.reshape(ridge_rfe_10.coef_, (-1,1)), columns=['Ridge_RFE_10'])],
         axis=1)

Ridge_coeff

#### Predictions with the above models

In [ ]:
# making predictions on normal Ridge model with 5 folds

y_train_pred = ridge.predict(X_train)
y_test_pred = ridge.predict(X_test)

print(f'Train R2 : {r2_score(y_train, y_train_pred)}')
print(f'Test R2 : {r2_score(y_test, y_test_pred)}')
print(f'Train Adj R2 : {1-(1-r2_score(y_train, y_train_pred))*(1011-1)/(1011-260-1)}')
print(f'Test Adj R2 : {1-(1-r2_score(y_test, y_test_pred))*(434-1)/(434-260-1)}')
print(f'Train MeanSqError : {mean_squared_error(y_train, y_train_pred)}')
print(f'Test MeanSqError : {mean_squared_error(y_test, y_test_pred)}\n')

# making predictions on RFE feature selected Ridge model with 5 folds

y_train_pred_rfe = ridge_rfe_5.predict(X_train_ridge_rfe)
y_test_pred_rfe = ridge_rfe_5.predict(X_test_ridge_rfe)

print(f'Train Ridge RFE R2_5 : {r2_score(y_train, y_train_pred_rfe)}')
print(f'Test Ridge RFE R2_5 : {r2_score(y_test, y_test_pred_rfe)}')
print(f'Train Adj R2_5 : {1-(1-r2_score(y_train, y_train_pred_rfe))*(1011-1)/(1011-76-1)}')
print(f'Test Adj R2_5 : {1-(1-r2_score(y_test, y_test_pred_rfe))*(434-1)/(434-76-1)}')
print(f'Train Ridge RFE R2_5 MeanSqError : {mean_squared_error(y_train, y_train_pred_rfe)}')
print(f'Test Ridge RFE R2_5 MeanSqError : {mean_squared_error(y_test, y_test_pred_rfe)}\n')

# making predictions on RFE feature selected Ridge model with 10 folds

y_train_pred_rfe_10 = ridge_rfe_10.predict(X_train_ridge_rfe)
y_test_pred_rfe_10 = ridge_rfe_10.predict(X_test_ridge_rfe)

print(f'Train Ridge RFE R2_10 : {r2_score(y_train, y_train_pred_rfe_10)}')
print(f'Test Ridge RFE R2_10 : {r2_score(y_test, y_test_pred_rfe_10)}')
print(f'Train Adj R2_10 : {1-(1-r2_score(y_train, y_train_pred_rfe_10))*(1011-1)/(1011-76-1)}')
print(f'Test Adj R2_10 : {1-(1-r2_score(y_test, y_test_pred_rfe_10))*(434-1)/(434-76-1)}')
print(f'Train Ridge RFE R2_10 MeanSqError : {mean_squared_error(y_train, y_train_pred_rfe_10)}')
print(f'Test Ridge RFE R2_10 MeanSqError : {mean_squared_error(y_test, y_test_pred_rfe_10)}\n')

## We can see that the models with selected features are performing much better than the model with all the
# features. Let's see next by calculating the VIF's and removing highly correlated features.

#### Checking VIF values , removing the features with very high values and running the models again

In [ ]:
# calculating VIF's to remove the highly correlated features

# We are using X_train_ridge_rfe dataframe with 130 features of training data for calculating VIF in this case

# Step 1. Run this code to get the VIF values and check it

vif_1 = pd.DataFrame()
vif_1['features'] = X_train_ridge_rfe.columns
vif_1['VIF'] = [variance_inflation_factor(X_train_ridge_rfe.values, i) 
                 for i in range(X_train_ridge_rfe.shape[1])] # calculating VIF's

vif_1.sort_values(by='VIF', ascending=False)

In [ ]:
## Step 2. Drop the feature with very high value, one by one and then after dropping the feature rerun step 1 
# then step 2 until your VIP score reaches below 5 for all the remaining features

# I already did this one by one and found the below features to be highly correlated which i will drop now

Remove_col_vif = ['BsmtFinSF2','CentralAir_N','SaleCondition_AdjLand','Street_Pave','MSZoning_RM',
                  'LandSlope_Gtl','Condition2_Norm','GrLivArea','PoolQC_NA','SaleCondition_Normal',
                  'YearBuilt','TotalBsmtSF','GarageYrBlt','SaleType_New','Exterior2nd_CmentBd','OverallQual',
                  'SaleType_WD','GarageCars','BedroomAbvGr','Exterior1st_VinylSd','1stFlrSF','CentralAir_Y',
                  'FullBath','KitchenQual_Gd','PavedDrive_Y','BsmtQual_Gd','Functional_Typ','GarageCond_TA',
                  'TotRmsAbvGrd','MSZoning_RL','OverallCond','HouseStyle_1.5Fin','BldgType_TwnhsE',
                    'BsmtExposure_NA','LandContour_Lvl','GarageArea','Condition1_Norm','LotArea','ExterCond_TA',
                  'YearRemodAdd','BsmtUnfSF','BsmtFinSF1','Exterior1st_Wd Sdng','Neighborhood_Somerst',
                 'HeatingQC_Ex']

# dropping all the highly correlated features which we got by running the above code one by one
X_train_ridge_rfe = X_train_ridge_rfe.drop(Remove_col_vif, axis=1)
X_test_ridge_rfe = X_test_ridge_rfe.drop(Remove_col_vif, axis=1)

In [ ]:
# Step 3. Run this finally to verify it

vif_1 = pd.DataFrame()
vif_1['features'] = X_train_ridge_rfe.columns
vif_1['VIF'] = [variance_inflation_factor(X_train_ridge_rfe.values, i) 
                 for i in range(X_train_ridge_rfe.shape[1])] # calculating VIF's

vif_1.sort_values(by='VIF', ascending=False)

In [ ]:
# checking shape of the df after dropping correlated features
print(X_train_ridge_rfe.shape)
print(X_test_ridge_rfe.shape)

In [ ]:
# fitting the Grid search model again to get the changed hyperparameter values
model_cv_rfe_5.fit(X_train_ridge_rfe, y_train)
model_cv_rfe_10.fit(X_train_ridge_rfe, y_train)

print(f'hyperparameter alpha with RFE feature selection and fold 5: {model_cv_rfe_5.best_params_}')
print(f'hyperparameter alpha with RFE feature selection and fold 10: {model_cv_rfe_10.best_params_}')

In [ ]:
# fitting the ridge model with alpha obtained above and checking the coefficients

alpha_ridge_rfe_5 = model_cv_rfe_5.best_params_['alpha']
alpha_ridge_rfe_10 = model_cv_rfe_10.best_params_['alpha']

ridge_rfe_5 = Ridge(alpha=alpha_ridge_rfe_5)
ridge_rfe_10 = Ridge(alpha=alpha_ridge_rfe_10)

ridge_rfe_5.fit(X_train_ridge_rfe, y_train)
ridge_rfe_10.fit(X_train_ridge_rfe, y_train)

Ridge_coeff_1 = pd.concat([pd.DataFrame(np.reshape(ridge_rfe_5.coef_, (-1,1)), columns=['Ridge_RFE_5']),
                         pd.DataFrame(np.reshape(ridge_rfe_10.coef_, (-1,1)), columns=['Ridge_RFE_10'])],
         axis=1)

Ridge_coeff_1

In [ ]:
# making predictions on RFE feature selected Ridge model with 5 folds

y_train_pred_rfe_1 = ridge_rfe_5.predict(X_train_ridge_rfe)
y_test_pred_rfe_1 = ridge_rfe_5.predict(X_test_ridge_rfe)

print(f'Train Ridge RFE R2_5 : {r2_score(y_train, y_train_pred_rfe_1)}')
print(f'Test Ridge RFE R2_5 : {r2_score(y_test, y_test_pred_rfe_1)}')
print(f'Train Adj R2_5 : {1-(1-r2_score(y_train, y_train_pred_rfe_1))*(1011-1)/(1011-86-1)}')
print(f'Test Adj R2_5 : {1-(1-r2_score(y_test, y_test_pred_rfe_1))*(434-1)/(434-86-1)}')
print(f'Train Ridge RFE R2_5 MeanSqError : {mean_squared_error(y_train, y_train_pred_rfe_1)}')
print(f'Test Ridge RFE R2_5 MeanSqError : {mean_squared_error(y_test, y_test_pred_rfe_1)}\n')

# making predictions on RFE feature selected Ridge model with 10 folds

y_train_pred_rfe_10_1 = ridge_rfe_10.predict(X_train_ridge_rfe)
y_test_pred_rfe_10_1 = ridge_rfe_10.predict(X_test_ridge_rfe)

print(f'Train Ridge RFE R2_10 : {r2_score(y_train, y_train_pred_rfe_10_1)}')
print(f'Test Ridge RFE R2_10 : {r2_score(y_test, y_test_pred_rfe_10_1)}')
print(f'Train Adj R2_10 : {1-(1-r2_score(y_train, y_train_pred_rfe_10_1))*(1011-1)/(1011-86-1)}')
print(f'Test Adj R2_10 : {1-(1-r2_score(y_test, y_test_pred_rfe_10_1))*(434-1)/(434-86-1)}')
print(f'Train Ridge RFE R2_10 MeanSqError : {mean_squared_error(y_train, y_train_pred_rfe_10_1)}')
print(f'Test Ridge RFE R2_10 MeanSqError : {mean_squared_error(y_test, y_test_pred_rfe_10_1)}\n')

#### we can see that both the above models R2 and Adj R2 are compromised, but with half number of features than before and hence models are much simpler and better than before. So, I tried using it by selecting more features 1st through VIF and then slowly tried reducing but our R2 & adj R2 got reduced by making our model simpler.

#### Train data residual analysis

In [ ]:
# Let's take the first model 'ridge_rfe_5' for further residual analysis

res_train = y_train - y_train_pred_rfe_1

# 1. error terms distribution plot
plt.figure(figsize=(18,18))

plt.subplot(221)
sns.distplot(res_train, bins = 20)
plt.title('Error Terms Distribution', fontsize = 20)            
plt.xlabel('Errors', fontsize = 18)  
## error terms seems to be normally distributed


# 2. look for patterns in residuals plotting a scatter plot
plt.subplot(222)
plt.scatter(np.arange(len(res_train)), res_train)
plt.plot([0]*len(res_train), color='red')
plt.title('Residuals Scatter plot', fontsize = 20)
## error terms seems to be randomly scattered


# 3. plotting qqplot of residuals of Model
sm.qqplot(res_train, fit=True, line='s', marker='.' )
plt.title('Q-Q plot of Residuals', fontsize=20)

# 4. plotting y train and y predicted to see how it fits the pattern

plt.figure(figsize=(16,10))
plt.plot(y_train, color='black')
plt.plot(y_train_pred_rfe_1, color='red')
plt.title('Train data Prediction', fontsize=20)
## we can see that it catches the patterns perfectly
plt.show()

#### Test data plot

In [ ]:
# Plotting the predictions on test data vs actual values

res_ridge = y_test - y_test_pred_rfe_1

# plotting y test and y predicted to see how it fits the pattern

plt.figure(figsize=(16,10))
plt.plot(y_test, color='black')
plt.plot(y_test_pred_rfe_1, color='red')
plt.title('Test data Prediction', fontsize=20)
plt.show()

### 4.2 Lasso Regression
#### Using RFE to get the important features using estimator=Lasso

In [ ]:
# using RFE to select features
lasso = Lasso()
lasso.fit(X_train, y_train)

# running the RFE, keeping the desired number of features to be half of the actual count of features
rfe_l = RFE(lasso,130)
rfe_l.fit(X_train, y_train)

In [ ]:
# creating df and extracting the features list

rfe_1asso_df = pd.DataFrame(list(zip(X_train.columns,rfe_l.support_,rfe_l.ranking_)),
             columns=('col','rfe support', 'rfe ranking')).sort_values(by='rfe ranking')

features_lasso_list = rfe_1asso_df[rfe_1asso_df['rfe support']==True]['col'].tolist()

#### Lasso Model with and without RFE selected features

In [ ]:
# finding the Lasso hyperparameter- best value of alpha needed for lasso model

# alpha values were selected from very low to higher value and after getting the best alpha, some more values 
# were inserted to make sure we get the best value

X_train_lasso_rfe = X_train[features_lasso_list]
X_test_lasso_rfe = X_test[features_lasso_list]

folds = [5, 10]
param = {'alpha': [0.00005, 0.0001, 0.00025, 0.0005, 0.001, 0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 
                   0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.05, 2.1, 2.15,
                   2.2, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000]}

model_cv_lasso = GridSearchCV(estimator=lasso, param_grid=param, scoring='neg_mean_absolute_error',
                       cv=folds[0], return_train_score=True, verbose=1)

model_cv_rfe_lasso_5 = GridSearchCV(estimator=lasso, param_grid=param, scoring='neg_mean_absolute_error',
                       cv=folds[0], return_train_score=True, verbose=1)

model_cv_rfe_lasso_10 = GridSearchCV(estimator=lasso, param_grid=param, scoring='neg_mean_absolute_error',
                       cv=folds[1], return_train_score=True, verbose=1)

model_cv_lasso.fit(X_train, y_train)
model_cv_rfe_lasso_5.fit(X_train_lasso_rfe, y_train)
model_cv_rfe_lasso_10.fit(X_train_lasso_rfe, y_train)

In [ ]:
# Printing the best hyperparameter alpha without feature selection
print(f'hyperparameter alpha without feature selection: {model_cv_lasso.best_params_}')

print(f'hyperparameter alpha with RFE feature selection and fold 5: {model_cv_rfe_lasso_5.best_params_}')

print(f'hyperparameter alpha with RFE feature selection and fold 10: {model_cv_rfe_lasso_10.best_params_}')

In [ ]:
# fitting the ridge model with alpha obtained above and checking the coefficients

alpha_lasso = model_cv_lasso.best_params_['alpha']
alpha_lasso_rfe_5 = model_cv_rfe_lasso_5.best_params_['alpha']
alpha_lasso_rfe_10 = model_cv_rfe_lasso_10.best_params_['alpha']

lasso = Lasso(alpha=alpha_lasso)
lasso_rfe_5 = Lasso(alpha=alpha_lasso_rfe_5)
lasso_rfe_10 = Lasso(alpha=alpha_lasso_rfe_10)

lasso.fit(X_train, y_train)
lasso_rfe_5.fit(X_train_lasso_rfe, y_train)
lasso_rfe_10.fit(X_train_lasso_rfe, y_train)

In [ ]:
# Coefficients of the models

Lasso_coeff = pd.concat([pd.DataFrame(np.reshape(lasso.coef_, (-1,1)), columns=['Lasso']), 
                         pd.DataFrame(np.reshape(lasso_rfe_5.coef_, (-1,1)), columns=['Lasso_RFE_5']),
                         pd.DataFrame(np.reshape(lasso_rfe_10.coef_, (-1,1)), columns=['Lasso_RFE_10'])],
         axis=1)

# we can see many features with '0' value which is what Lasso does
#Lasso_coeff

#### Predictions with above models

In [ ]:
# making predictions on normal Lasso model with 5 folds

y_train_pred_lasso = lasso.predict(X_train)
y_test_pred_lasso = lasso.predict(X_test)

print(f'Train R2 : {r2_score(y_train, y_train_pred_lasso)}')
print(f'Test R2 : {r2_score(y_test, y_test_pred_lasso)}')
print(f'Train Adj R2 : {1-(1-r2_score(y_train, y_train_pred_lasso))*(1011-1)/(1011-260-1)}')
print(f'Test Adj R2 : {1-(1-r2_score(y_test, y_test_pred_lasso))*(434-1)/(434-260-1)}')
print(f'Train MeanSqError : {mean_squared_error(y_train, y_train_pred_lasso)}')
print(f'Test MeanSqError : {mean_squared_error(y_test, y_test_pred_lasso)}\n')

# making predictions on RFE feature selected Lasso model with 5 folds

y_train_pred_lasso_rfe = lasso_rfe_5.predict(X_train_lasso_rfe)
y_test_pred_lasso_rfe = lasso_rfe_5.predict(X_test_lasso_rfe)

print(f'Train Lasso RFE R2_5 : {r2_score(y_train, y_train_pred_lasso_rfe)}')
print(f'Test Lasso RFE R2_5 : {r2_score(y_test, y_test_pred_lasso_rfe)}')
print(f'Train Lasso Adj R2_5 : {1-(1-r2_score(y_train, y_train_pred_lasso_rfe))*(1011-1)/(1011-130-1)}')
print(f'Test Lasso Adj R2_5 : {1-(1-r2_score(y_test, y_test_pred_lasso_rfe))*(434-1)/(434-130-1)}')
print(f'Train Lasso RFE R2_5 MeanSqError : {mean_squared_error(y_train, y_train_pred_lasso_rfe)}')
print(f'Test Lasso RFE R2_5 MeanSqError : {mean_squared_error(y_test, y_test_pred_lasso_rfe)}\n')

# making predictions on RFE feature selected Lasso model with 10 folds

y_train_pred_lasso_rfe_10 = lasso_rfe_10.predict(X_train_lasso_rfe)
y_test_pred_lasso_rfe_10 = lasso_rfe_10.predict(X_test_lasso_rfe)

print(f'Train Lasso RFE R2_10 : {r2_score(y_train, y_train_pred_lasso_rfe_10)}')
print(f'Test Lasso RFE R2_10 : {r2_score(y_test, y_test_pred_lasso_rfe_10)}')
print(f'Train Lasso Adj R2_10 : {1-(1-r2_score(y_train, y_train_pred_lasso_rfe_10))*(1011-1)/(1011-130-1)}')
print(f'Test Lasso Adj R2_10 : {1-(1-r2_score(y_test, y_test_pred_lasso_rfe_10))*(434-1)/(434-130-1)}')
print(f'Train Lasso RFE R2_10 MeanSqError : {mean_squared_error(y_train, y_train_pred_lasso_rfe_10)}')
print(f'Test Lasso RFE R2_10 MeanSqError : {mean_squared_error(y_test, y_test_pred_lasso_rfe_10)}\n')

## We can see that the models with selected features are performing worse than the model with all the
# features but this model Adjusted R2 and R2 gap is too wide. So, all the models we used here are not upto 
# the mark. Let's see next by calculating the VIF's for the model with all the features and removing 
# highly correlated features.

#### Since our base  model for Lasso is working much better than the other ones, we will be continuing with that model.
#### Checking VIF values , removing the features with very high values and running the models again

In [ ]:
# calculating VIF's to remove the highly correlated features

# We are using full dataframe with full features of training data for calculating VIF in this case

# Step 1. Run this code to get the VIF values and check it

vif_lasso = pd.DataFrame()
vif_lasso['features'] = X_train.columns
vif_lasso['VIF'] = [variance_inflation_factor(X_train.values, i) 
                 for i in range(X_train.shape[1])] # calculating VIF's

# getting the VIF's sorted, easier to check
vif_lasso.sort_values(by='VIF', ascending=False)

In [ ]:
## Step 2. Drop the feature with very high value, one by one and then after dropping the feature rerun step 1 
# then step 2 until your VIP score reaches below 5 for all the remaining features

# I already did this one by one and found the below features to be highly correlated which i will drop now

Remove_lasso_vif = ['Exterior2nd_Wd Shng','SaleCondition_Partial','Exterior1st_BrkFace','MoSold_4','HouseStyle_2Story',
                   'RoofStyle_Hip','Fence_MnPrv','PoolQC_Gd','BldgType_Duplex','PavedDrive_Y','BldgType_1Fam',
                   'GarageQual_Fa','Condition1_others','GarageType_NA','GarageFinish_NA','GarageFinish_RFn',
                   'Neighborhood_SawyerW','Functional_Min2','FireplaceQu_Fa','KitchenQual_Gd','Electrical_SBrkr',
                   'BsmtQual_NA','LandSlope_Mod','CentralAir_Y','LotConfig_FR3','HeatingQC_TA','GarageType_Detchd',
                   'LandContour_Low','Heating_GasA','LotShape_Reg','BsmtFinType2_NA','Alley_Pave','RoofMatl_CompShg',
                   'MSZoning_RM','BsmtFinType1_NA','BsmtFinType1_LwQ','Condition2_others','MSSubClass_80',
                   'BsmtQual_Ex','MSSubClass_190','ExterQual_Gd','BsmtCond_NA','GarageCond_TA','ExterCond_Gd',
                   'MasVnrType_BrkFace','MiscFeature_Shed','Street_Grvl','YrSold_2010','BsmtExposure_NA',
                   'Foundation_PConc','BsmtFinSF2','SaleType_others','GarageCond_NA','Utilities_AllPub','BsmtCond_TA',
                   'GrLivArea','PoolQC_NA','Street_Pave','BsmtFinType2_Unf','SaleCondition_Normal','TotalBsmtSF',
                   'LotConfig_Inside','YearBuilt','Condition2_Norm','BsmtExposure_No','MSSubClass_20','GarageYrBlt',
                   'LandContour_Lvl','HouseStyle_1Story','OverallQual','Alley_NA','Exterior1st_CemntBd',
                   'GarageQual_TA','GarageCars','BedroomAbvGr','Exterior2nd_MetalSd','1stFlrSF','SaleType_WD',
                   'Exterior1st_VinylSd','Functional_Typ','LandSlope_Gtl','FireplaceQu_NA','FullBath','MiscFeature_NA',
                   'MSZoning_RL','OverallCond','Condition1_Norm','TotRmsAbvGrd','MSSubClass_50','Exterior2nd_VinylSd',
                   'BldgType_TwnhsE','GarageArea','BsmtQual_TA','2ndFlrSF','LotArea','ExterCond_TA','YearRemodAdd',
                   'BsmtFinSF1','Exterior1st_HdBoard','Fence_NA','Fireplaces','ExterQual_TA','BsmtFinType1_Unf',
                   'GarageType_Attchd','MasVnrType_None','Neighborhood_Somerst','RoofStyle_Gable','Foundation_CBlock',
                   'Exterior2nd_Wd Sdng','LotFrontage','BsmtUnfSF','HeatingQC_Ex']


# dropping all the highly correlated features which we got by running the above code one by one
X_train_lasso_VIF = X_train.drop(Remove_lasso_vif, axis=1)
X_test_lasso_VIF = X_test.drop(Remove_lasso_vif, axis=1)

In [ ]:
# Step 3. Run this finally to verify it

vif_lasso = pd.DataFrame()
vif_lasso['features'] = X_train_lasso_VIF.columns
vif_lasso['VIF'] = [variance_inflation_factor(X_train_lasso_VIF.values, i) 
                 for i in range(X_train_lasso_VIF.shape[1])] # calculating VIF's

# getting the VIF's sorted, easier to check
vif_lasso.sort_values(by='VIF', ascending=False)

In [ ]:
# checking shape of the df after dropping correlated features
print(X_train_lasso_VIF.shape)
print(X_test_lasso_VIF.shape)

In [ ]:
# fitting the Grid search model again to get the changed hyperparameter values
model_cv_lasso_vif = GridSearchCV(estimator=lasso, param_grid=param, scoring='neg_mean_absolute_error',
                       cv=folds[0], return_train_score=True, verbose=1)

model_cv_lasso_vif.fit(X_train_lasso_VIF, y_train)

print(f'hyperparameter alpha with RFE feature selection and fold 5: {model_cv_lasso_vif.best_params_}')

In [ ]:
# fitting the lasso model with alpha obtained above and checking the coefficients

alpha_lasso_vif = model_cv_lasso_vif.best_params_['alpha']

lasso_vif = Lasso(alpha=alpha_lasso_vif)

lasso_vif.fit(X_train_lasso_VIF, y_train)

Lasso_coeff_vif = pd.concat([pd.DataFrame(np.reshape(lasso_vif.coef_, (-1,1)), columns=['Lasso_VIF'])],
                             axis=1)

Lasso_coeff_vif

In [ ]:
# code used in question 1 and 3
#dropcol= ['PoolQC_Ex', 'MasVnrArea','OpenPorchSF','Neighborhood_NoRidge','HouseStyle_2.5Unf'] 
#ques_df = X_train_lasso_VIF.drop(dropcol, axis=1)
#lasso_vif.fit(ques_df, y_train)
#Lasso_coeff_vif_1 = pd.concat([pd.DataFrame(np.reshape(lasso_vif.coef_, (-1,1)), columns=['Lasso_VIF']),
#                            pd.DataFrame((X_train_lasso_VIF.columns), columns=['feature name'])],
#                            axis=1)
#Lasso_coeff_vif_1.sort_values(by='Lasso_VIF', ascending=False)

In [ ]:
# making predictions on Lasso VIF feature selected model with 5 folds

y_train_pred_vif = lasso_vif.predict(X_train_lasso_VIF)
y_test_pred_vif = lasso_vif.predict(X_test_lasso_VIF)

y_train_pred_vif = np.reshape(y_train_pred_vif, (-1,1))
y_test_pred_vif = np.reshape(y_test_pred_vif, (-1,1))

print(f'Train Lasso VIF R2 : {r2_score(y_train, y_train_pred_vif)}')
print(f'Test Lasso VIF R2 : {r2_score(y_test, y_test_pred_vif)}')
print(f'Train VIF Adj R2 : {1-(1-r2_score(y_train, y_train_pred_vif))*(1011-1)/(1011-148-1)}')
print(f'Test VIF Adj R2 : {1-(1-r2_score(y_test, y_test_pred_vif))*(434-1)/(434-148-1)}')
print(f'Train Lasso VIF R2 MeanSqError : {mean_squared_error(y_train, y_train_pred_vif)}')
print(f'Test Lasso VIF R2 MeanSqError : {mean_squared_error(y_test, y_test_pred_vif)}\n')

#### After observing all the predictions of model with all the features, features reduced by using RFE and VIF, we can say that the last model is performing better.

#### Train data residual analysis

In [ ]:
# Let's take the last model 'lasso_vif' which we developed using VIF for further residual analysis

res_lasso_train = y_train - y_train_pred_vif

# 1. error terms distribution plot
plt.figure(figsize=(18,18))

plt.subplot(221)
sns.distplot(res_lasso_train, bins = 20)
plt.title('Error Terms Distribution', fontsize = 20)            
plt.xlabel('Errors', fontsize = 18)  
## error terms seems to be normally distributed


# 2. look for patterns in residuals plotting a scatter plot
plt.subplot(222)
plt.scatter(np.arange(len(res_lasso_train)), res_lasso_train)
plt.plot([0]*len(res_lasso_train), color='red')
plt.title('Residuals Scatter plot', fontsize = 20)
## error terms seems to be randomly scattered


# 3. plotting qqplot of residuals of Model
sm.qqplot(res_lasso_train, fit=True, line='s', marker='.' )
plt.title('Q-Q plot of Residuals', fontsize=20)

# 4. plotting y train and y predicted to see how it fits the pattern

plt.figure(figsize=(16,10))
plt.plot(y_train, color='black')
plt.plot(y_train_pred_vif, color='red')
plt.title('Train data Prediction', fontsize=20)
## we can see that it's not catching the patterns perfectly as Ridge models were catching
plt.show()

#### Test data predictions

In [ ]:
# Plotting the predictions on test data vs actual values

# plotting y test and y predicted to see how it fits the pattern

plt.figure(figsize=(16,10))
plt.plot(y_test, color='black')
plt.plot(y_test_pred_vif, color='red')
plt.title('Test data Prediction', fontsize=20)
plt.show()

### It's very clear that Ridge models are performing better than Lasso models. We can choose the one where feature selection was done through RFE and that model has high R2 and closer adjusted R2. But if we want high bias model with low variance we can select Ridge model where feature selection was done through RFE AND VIF both. 